<a href="https://colab.research.google.com/github/chiyeon01/ML_CONTEST/blob/main/DACON/%EC%B1%84%EB%AC%B4%20%EB%B6%88%EC%9D%B4%ED%96%89%20%EC%97%AC%EB%B6%80%20%EC%98%88%EC%B8%A1%20%ED%95%B4%EC%BB%A4%ED%86%A4%3A%20%EB%B6%88%EC%9D%B4%ED%96%89%EC%9D%98%20%EC%A7%95%ED%9B%84%EB%A5%BC%20%EC%B0%BE%EC%95%84%EB%9D%BC!/Voting_Bagging%ED%99%9C%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score

from google.colab import drive

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/Colab Notebooks/z.대회 & 프로젝트/2025_채무 불이행 여부 예측 해커톤_DACON/open.zip'

Archive:  /content/drive/MyDrive/Colab Notebooks/z.대회 & 프로젝트/2025_채무 불이행 여부 예측 해커톤_DACON/open.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
train_df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv('/content/test.csv')
sample_df = pd.read_csv('/content/sample_submission.csv')

In [ ]:
data_df = pd.concat([train_df, test_df])

In [ ]:
encoder = LabelEncoder()
object_columns = data_df.dtypes[data_df.dtypes == 'object'].index.tolist()

for column in object_columns:
    data_df[column] = encoder.fit_transform(data_df[column])

In [ ]:
train_df = data_df[~data_df['채무 불이행 여부'].isnull()]
test_df = data_df[data_df['채무 불이행 여부'].isnull()]

In [ ]:
# features와 targets를 나누기.
features = train_df.iloc[:, :-1]
targets = train_df.iloc[:, -1]

In [ ]:
features.drop(labels=['UID'], axis=1, inplace=True)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(features, targets, test_size=0.15, stratify=targets, random_state=42)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(8500, 16) (1500, 16) (8500,) (1500,)


In [ ]:
voting_clf = VotingClassifier(
    estimators = [
        ('lr', LogisticRegression(random_state=42)),
        ('rf', RandomForestClassifier(max_depth=3, random_state=42)),
        ('svc', SVC(probability=True, random_state=42)),
        ('lgb_clf', LGBMClassifier(random_state=42)),
        ('xgb_clf', XGBClassifier(random_state=42))
    ],
    voting='soft'
)

In [ ]:
voting_clf.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 2900, number of negative: 5600
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001051 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1861
[LightGBM] [Info] Number of data points in the train set: 8500, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.341176 -> initscore=-0.658056
[LightGBM] [Info] Start training from score -0.658056


VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('rf',
                              RandomForestClassifier(max_depth=3,
                                                     random_state=42)),
                             ('svc', SVC(probability=True, random_state=42)),
                             ('lgb_clf', LGBMClassifier(random_state=42)),
                             ('xgb_clf',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytre...
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=42, ...))],
                 voting='soft')

In [ ]:
voting_clf.score(X_val, y_val)

0.7166666666666667

In [ ]:
pred = voting_clf.predict(X_val)
print(roc_auc_score(pred, y_val))

0.733099679220625


In [ ]:
'''
voting 방식을 이용하니 기존 DecisionTree와 lgb_clf의 단독적인 성능보다 효과가 좋음.
bagging은 randomforest방식에 녹아 들어가니 따로 시험은 안해봄.
'''

# 제출 파일 만들기.

In [ ]:
voting_clf.fit(features, targets)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 3412, number of negative: 6588
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001487 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1865
[LightGBM] [Info] Number of data points in the train set: 10000, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.341200 -> initscore=-0.657951
[LightGBM] [Info] Start training from score -0.657951


VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('rf',
                              RandomForestClassifier(max_depth=3,
                                                     random_state=42)),
                             ('svc', SVC(probability=True, random_state=42)),
                             ('lgb_clf', LGBMClassifier(random_state=42)),
                             ('xgb_clf',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytre...
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=42, ...))],
                 voting='soft')

In [ ]:
test_df.drop(labels='채무 불이행 여부', axis=1, inplace=True)

<ipython-input-20-94d585deaab1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.drop(labels='채무 불이행 여부', axis=1, inplace=True)


In [ ]:
pred = voting_clf.predict_proba(test_df.iloc[:, 1:])

In [ ]:
sample_df['채무 불이행 확률'] = pred[:, 1]

In [ ]:
sample_df

,UID,채무 불이행 확률
0,TEST_0000,0.425253
1,TEST_0001,0.298957
2,TEST_0002,0.303994
3,TEST_0003,0.262546
4,TEST_0004,0.299216
...,...,...
2057,TEST_2057,0.218419
2058,TEST_2058,0.245730
2059,TEST_2059,0.381890
2060,TEST_2060,0.355950


In [ ]:
sample_df.to_csv('/content/drive/MyDrive/Colab Notebooks/z.대회 & 프로젝트/2025_채무 불이행 여부 예측 해커톤_DACON/Voting_pred.csv', index=False)